## IMDB Movie Reviews using Word2Vec in Gensim library.

## [ Please star/upvote if u like it. ]

#### IMPORTING MODULES

In [1]:
import nltk
nltk.download('all',halt_on_error=False)

!pip install gensim

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [0]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
% matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# BeautifulSoup libraray
from bs4 import BeautifulSoup 

import re # regex

#model_selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#evaluation
from sklearn.metrics import accuracy_score,roc_auc_score 
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix

#preprocessing scikit
from sklearn.preprocessing import MinMaxScaler,StandardScaler,Imputer,LabelEncoder

#classifiaction.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
 
#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence

#gensim w2v
#word2vec
from gensim.models import Word2Vec  

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


#### LOADING THE DATASET

In [0]:
train = pd.read_csv(r'drive/Colab Notebooks/IMDB Movie Review Kaggle/Keras/labeledTrainData.tsv', header=0,\
                    delimiter="\t", quoting=3)

test = pd.read_csv(r'drive/Colab Notebooks/IMDB Movie Review Kaggle/Keras/testData.tsv',header=0,\
                    delimiter="\t", quoting=3)

In [6]:
print("the train shape is : ",train.shape)
print("the test shape is : ",test.shape)

the train shape is :  (25000, 3)
the test shape is :  (25000, 2)


In [0]:
df=train.copy()

In [8]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [0]:
df.drop('id',axis=1,inplace=True)

In [10]:
df.head()

,sentiment,review
0,1,"""With all this stuff going down at the moment ..."
1,1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,0,"""The film starts with a manager (Nicholas Bell..."
3,0,"""It must be assumed that those who praised thi..."
4,1,"""Superbly trashy and wondrously unpretentious ..."


In [11]:
df['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [12]:
df['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [13]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [14]:
test['review'][0]

'"Naturally in a film who\'s main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce\'s short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."'

#### DATA CLEANING & PRE-PROCESSING

In [0]:
def clean_reviews(review): # actualy receiving individual sentences in this case.
    
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    
    # 2. Retaining only alphabets.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    
    # 3. Converting to lower case and splitting
    word_tokens= review_text.lower().split()
    
    # 4. Remove stopwords
    stop_words= set(stopwords.words("english"))     
    word_tokens= [w for w in word_tokens if not w in stop_words]
    
    cleaned_review=" ".join(word_tokens)
    return cleaned_review
  
  

In [18]:
# time taking cell.
all_reviews=df['review']+test['review']
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences=[]
sum=0
for review in all_reviews:
  sents=tokenizer.tokenize(review.strip())
  sum+=len(sents)
  for sent in sents:
    cleaned_sent=clean_reviews(sent)
    sentences.append(cleaned_sent.split())
print(sum)
print(len(sentences))
    

523328
523328


Note that the Word2Vec module in Gensim expects a list of lists where each inner list is itself a tokenized sentence.

Therefore this part of code basically iterates over all reviews in train and test sets and then tokenizes them to sentences.

Then after this each sentence is passed for data cleaning and then appended to the main list (that passed to gensim w2v model) by splitting the sentence into individula tokens or words.

**Note that if u first clean the review then all the period (.) will be removed and hence the review will not be broken into sentences properly by the tokenizer.**

In [19]:
for i in range(10): # just printing few sentences.
  print(sentences[i])

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker']
['maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent']
['moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released']
['subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring']
['may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lor

#### BUILDING THE WORD2VEC WORD EMBEDDINGS AND TRAINING

In [0]:
w2v_model=Word2Vec(sentences,size=300,window=10,min_count=40) # try to tune parameters.

In [21]:
w2v_model.train(sentences,total_examples=len(sentences),epochs=10)

(51821730, 59070630)

#### word vector of a particular word in vocab

In [22]:
w2v_model.wv.get_vector('movie')

array([ 6.5689415e-01,  2.2706357e-01,  2.1373306e-01, -2.9602724e-01,
       -8.6462861e-01, -4.1036540e-01,  4.9871609e-02, -9.1681063e-01,
        1.6984144e-02, -2.8281337e-01, -5.8414358e-01,  1.0722756e+00,
        5.8135217e-01,  1.3113582e+00, -4.6099809e-01,  5.6005418e-01,
        3.4210420e-01, -1.8675509e-01, -1.9541378e+00,  5.3132838e-01,
       -1.4399172e-01, -3.8046387e-01,  4.1382623e-01, -6.2318140e-01,
        1.9824916e-01, -2.8089488e-01, -1.8464778e-01, -1.9997297e-01,
        1.0760150e+00,  6.7964947e-01,  7.7137375e-01,  3.5987607e-01,
       -1.0876212e+00, -1.6859546e-01, -6.1458534e-01,  3.0477729e-01,
        6.6191882e-01,  4.0804434e-01,  1.9429240e+00,  9.9214280e-01,
       -6.5652215e-01,  8.9349765e-01,  9.5025457e-02, -7.8943324e-01,
        3.5290802e-01, -9.8253500e-01,  7.5649363e-01,  6.3348430e-01,
       -1.9864178e+00,  5.8297092e-01,  3.8680595e-01, -1.3842666e+00,
       -2.5695274e+00,  1.0695862e+00,  5.6507033e-01,  5.4342049e-01,
      

#### similar words to a given word

In [23]:
w2v_model.wv.most_similar('good')

[('decent', 0.617585301399231),
 ('great', 0.5900850892066956),
 ('bad', 0.582518458366394),
 ('ok', 0.5118921995162964),
 ('nice', 0.48266398906707764),
 ('alright', 0.4559839069843292),
 ('lousy', 0.445183664560318),
 ('okay', 0.44331270456314087),
 ('excellent', 0.4266152083873749),
 ('really', 0.42602548003196716)]

#### simalarity between two given words in vocab

In [24]:

w2v_model.wv.similarity('movie','film')

0.7646104

In [25]:
# random words
w2v_model.wv.similarity('movie','gannon')

-0.28946522

#### CREATING THE DOCUMENT FEATURE VECTORS

#### Now we have vectors for all the words in the training vocab. Now we start building the document vectors. We can have different ways to construct the document feature vectors from the word embeddings given by w2v model eg (avg word vectors ,sum of word vectors ,doc2vec etc...)

Check out **[this](https://stackoverflow.com/questions/29760935/how-to-get-vector-for-a-sentence-from-the-word2vec-of-tokens-in-sentence)** discussion on SO.

Below I have implemented the document vectors by averaging the word vectors of all the words in the respective document. This is done by the **make_doc_vectors** function.

In [0]:
df['review']=df['review'].apply(clean_reviews)
test['review']=test['review'].apply(clean_reviews)

In [0]:
def make_doc_vectors(review):
  doc_vec=np.zeros(300,dtype='float64') # initialize with zeros.
  words=review.split()
  for word in words: # iterate over all words.
    if word in w2v_model.wv.vocab:
      word_vec=w2v_model.wv.get_vector(word) # if word in vocab add its word vector to our doc vector
      doc_vec=doc_vec+word_vec
  return doc_vec/len(words)  # returning the averaged word vector ie the 'doc vector' for us.  

In [0]:
doc_vectors_train=[]
for review in df['review']:
  doc_vectors_train.append(make_doc_vectors(review))
doc_vectors_train=np.array(doc_vectors_train)

Now **doc_vectors_train** is the kind off 'feature matrix' and **Y** contains the corressponding sentiment.

In [36]:
doc_vectors_train.shape

(25000, 300)

In [0]:
Y=np.array(list(df['sentiment']))

#### SPLITTING IN TRAIN & VAL SETS.

In [0]:
x_train,x_test,y_train,y_test=train_test_split(doc_vectors_train,Y,test_size=0.20,random_state=42)

#### MODELLING ALGOS.

Different modelling algos have been used. The parametres can be tuned for better performance using GridSearchCV (say).

#### Logistic Regression

In [43]:
clf_lr=LogisticRegression(C=10)
clf_lr.fit(x_train,y_train)
pred=clf_lr.predict(x_test)
print(accuracy_score(pred,y_test))

0.878


#### Support Vector Machine ( SVM )

In [44]:
clf_svm=SVC()
clf_svm.fit(x_train,y_train)
pred=clf_svm.predict(x_test)
print(accuracy_score(pred,y_test))

0.878


#### Gradient Boosting 

In [45]:
clf_gb=GradientBoostingClassifier()
clf_gb.fit(x_train,y_train)
pred=clf_gb.predict(x_test)
print(accuracy_score(pred,y_test))

0.853


#### Random Forest Classifier

In [46]:
clf_rf=RandomForestClassifier()
clf_rf.fit(x_train,y_train)
pred=clf_rf.predict(x_test)
print(accuracy_score(pred,y_test))

0.7934


## THE END!!!

## [Please star/upvote if u liked it.;) ]